In [ ]:
import os
import mne
import pyxdf
import numpy as np
from mne.preprocessing import ICA
import matplotlib.pyplot as plt
from mne_icalabel import label_components
plt.style.use('default')
%matplotlib qt
from utils import read_data, calculate_power_spectrum, plot_power_spectrum
import glob
import autoreject
from mne.io import read_raw_eeglab

In [ ]:
set_file = "ex1-eeglab.set"
raw_eeglab = read_raw_eeglab(set_file, preload=True)
notch_freq = 60  # Notch frequency in Hz
sampling_rate = raw_eeglab.info['sfreq']  # Sampling frequency in Hz
raw_eeglab = raw_eeglab.notch_filter(
    np.arange(notch_freq, sampling_rate/2, notch_freq), picks='eeg')

In [ ]:
raw_eeglab.compute_psd(fmax=62).plot()

In [ ]:
power, freqs = calculate_power_spectrum(
    raw_eeglab, fmin=2, fmax=62, method='welch')
plot_power_spectrum(power, freqs, average_axis=0)

In [3]:
exp_path = os.path.join("exp_data","02_Experimental")
control_path = os.path.join("exp_data","01_Control")
glob_pattern = os.path.join("**","*.xdf")

exp_files = glob.glob(os.path.join(exp_path,glob_pattern),recursive=True)
control_files = glob.glob(os.path.join(control_path,glob_pattern),recursive=True)[:6]
# exp_mapping, exp_common = get_event_names(control_files)
# CTRL03-sub-129059-old error

In [5]:
exp_files

['exp_data/02_Experimental/EXP01/sub-323706/sub-323706_task-hearing_run-001.xdf',
 'exp_data/02_Experimental/EXP06/sub-680171/sub-680171_task-hearing_run-001.xdf',
 'exp_data/02_Experimental/EXP07/sub-446052/sub-446052_task-hearing_run-001.xdf',
 'exp_data/02_Experimental/EXP05/sub-638390/sub-638390_task-hearing_run-001.xdf',
 'exp_data/02_Experimental/EXP02/sub-081795/sub-081795_task-hearing_run-001.xdf',
 'exp_data/02_Experimental/EXP03/sub-198651/sub-198651_task-hearing_run-001.xdf',
 'exp_data/02_Experimental/EXP04/sub-767328/sub-767328_task-hearing_run-001.xdf']

In [4]:
raw, events, mapping = read_data(
    exp_files[0], bandpass={'low': 2, 'high': 50}, flat_voltage=0.1, notch_freq=60)
montage = mne.channels.read_custom_montage(
    'ceegrid_coords.csv', coord_frame='head')
raw.set_montage(montage)
# raw.export('EXP01.set', fmt='eeglab')

IndexError: list index out of range

In [ ]:
power, freqs = calculate_power_spectrum(raw, fmin=2, fmax=62, method='welch')
plot_power_spectrum(power, freqs, average_axis=0)

In [ ]:
raw, events, mapping = read_data(
    exp_files[2], bandpass={'low': 2, 'high': 50}, flat_voltage=0.1)
montage = mne.channels.read_custom_montage(
    'ceegrid_coords.csv', coord_frame='head')
raw.set_montage(montage)
raw.interpolate_bads(reset_bads=True)
event_type = 'ast_stim'
group = 'all'
epoch_trigger = mne.Eposchs(
    raw, events, event_id=mapping[event_type]['trigger'], tmin=-0.5, tmax=4, preload=True, baseline=(None, 0))
epoch_neutral = mne.Epochs(
    raw, events, event_id=mapping[event_type]['neutral'], tmin=-0.5, tmax=4, preload=True, baseline=(-0.5, 0))
epoch_all = mne.Epochs(
    raw, events, event_id=mapping[event_type]['all'], tmin=-0.5, tmax=4, preload=True)

In [ ]:
epoch = mne.Epochs(
    raw, events, event_id={'neutral': 1, 'trigger': 2}, tmin=-0.2, tmax=4)
epoch

In [ ]:
epoch_all

In [ ]:
raw.plot()

In [ ]:
power, freqs = calculate_power_spectrum(
    raw, fmin=2, fmax=50, method='welch')
plot_power_spectrum(power, freqs, average_axis=(0))

In [ ]:
power.shape

In [ ]:
plot_power_spectrum(power, freqs)

In [ ]:
ar = autoreject.AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11,
                           n_jobs=1, verbose=True)
ar.fit(epoch_all)  # fit on a few epochs to save time
epochs_ar, reject_log = ar.transform(epoch_all, return_log=True)

In [ ]:
power_ar, freqs_ar = calculate_power_spectrum(epochs_ar, fmin=2, fmax=50)
plot_power_spectrum(power_ar, freqs_ar)

In [ ]:
left_ear = ['L1', 'L2', 'L4', 'L5', 'L7', 'L8', 'L9', 'L10']
right_ear = ['R1', 'R2', 'R4', 'R5', 'R7', 'R8', 'R9', 'R10']

evokeds = dict(trigger=epoch_trigger.average(), neutral=epoch_neutral.average())
picks = left_ear
mne.viz.plot_compare_evokeds(evokeds, picks=picks, combine="mean")

In [ ]:
freqs = np.logspace(*np.log10([1, 20]), num=8) # alpha band frequencies
n_cycles = freqs / 2.0  # different number of cycle per frequency
power, itc = exp_epoch['ast_stim-paper_crunching'].compute_tfr(
    method="morlet",
    freqs=freqs,
    n_cycles=n_cycles,
    average=True,
    return_itc=True,
    decim=3,
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

channels = power.ch_names
n_channels = len(channels)
n_cols = 4
n_rows = int(np.ceil(n_channels / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 3 * n_rows))
axes = axes.flatten()

# Normalize each channel's power between 0 and 1
for idx, ch in enumerate(channels):
    data = power.data[idx]
    data_norm = (data - data.min()) / (data.max() - data.min() + 1e-12)
    im = axes[idx].imshow(
        data_norm,
        aspect='auto',
        origin='lower',
        extent=[power.times[0], power.times[-1], power.freqs[0], power.freqs[-1]],
        vmin=0,
        vmax=1,
        cmap='Reds'
    )
    axes[idx].set_title(ch)
    axes[idx].set_ylabel('Freq (Hz)')
    axes[idx].set_xlabel('Time (s)')

# Hide any unused subplots
for ax in axes[n_channels:]:
    ax.axis('off')

# Add a single colorbar to the right
fig.subplots_adjust(right=0.88)
cbar_ax = fig.add_axes([0.90, 0.15, 0.02, 0.7])
plt.colorbar(im, cax=cbar_ax)

plt.tight_layout(rect=[0, 0, 0.88, 1])

In [ ]:
import numpy as np

# Define frequency bands (Hz)
bands = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 45)
}

# Get frequency indices for each band
band_indices = {
    band: np.where((power.freqs >= low) & (power.freqs < high))[0]
    for band, (low, high) in bands.items()
}

# Calculate normalized average power per band per channel
band_power_norm = {}
for band, idxs in band_indices.items():
    band_power_norm[band] = []
    for ch in range(power.data.shape[0]):
        data = power.data[ch]
        # Normalize the TFR for this channel
        data_norm = (data - data.min()) / (data.max() - data.min() + 1e-12)
        # Average over band freqs and all times
        band_avg = data_norm[idxs, :].mean()
        band_power_norm[band].append(band_avg)
    band_power_norm[band] = np.array(band_power_norm[band])

# Print normalized average power for each band and channel
for band in bands:
    print(f"\n{band.capitalize()} band normalized average power per channel:")
    for ch, avg in zip(power.ch_names, band_power_norm[band]):
        print(f"  {ch}: {avg:.4f}")

# Example: Compute alpha/beta ratio per channel
alpha = band_power_norm['alpha']
beta = band_power_norm['beta']
alpha_beta_ratio = alpha / (beta + 1e-12)  # Avoid division by zero

print("\nAlpha/Beta ratio per channel:")
for ch, ratio in zip(power.ch_names, alpha_beta_ratio):
    print(f"  {ch}: {ratio:.4f}")

# You can compute other ratios similarly, e.g., theta/alpha, beta/gamma, etc.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Bar plot: Normalized average power per band per channel
bands_list = list(band_power_norm.keys())
channels = power.ch_names
x = np.arange(len(channels))

fig, ax = plt.subplots(figsize=(14, 6))
width = 0.15

for i, band in enumerate(bands_list):
    if band == 'delta':
        continue
    ax.bar(x + i * width, band_power_norm[band], width, label=band.capitalize())

ax.set_xticks(x + width * (len(bands_list) - 1) / 2)
ax.set_xticklabels(channels, rotation=45)
ax.set_ylabel('Normalized Average Power')
ax.set_title('Normalized Average Power per Band per Channel')
ax.legend()
plt.tight_layout()
plt.show()

# Bar plot: Alpha/Beta ratio per channel
fig, ax = plt.subplots(figsize=(14, 4))
ax.bar(channels, alpha_beta_ratio, color='orange')
ax.set_ylabel('Alpha/Beta Ratio')
ax.set_title('Alpha/Beta Ratio per Channel')
ax.set_xticklabels(channels, rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
control_ab = alpha_beta_ratio

In [ ]:
channels = power.ch_names
x = np.arange(len(channels))
width = 0.35

fig, ax = plt.subplots(figsize=(14, 4))
ax.bar(x + width/2, control_ab, width, color='green', label='Control')
ax.bar(x - width/2, alpha_beta_ratio, width, color='orange', label='Experimental')
ax.set_ylabel('Alpha/Beta Ratio')
ax.set_title('Alpha/Beta Ratio per Channel')
ax.set_xticks(x)
ax.set_xticklabels(channels, rotation=45)
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
power.plot(baseline=(-0.2, 0), mode='mean', colorbar=True)

In [ ]:
import matplotlib.pyplot as plt

channels = power.ch_names
n_channels = len(channels)
n_cols = 4  # You can adjust this for your preferred layout
n_rows = int(np.ceil(n_channels / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 3 * n_rows))
axes = axes.flatten()

for idx, ch in enumerate(channels):
    power.plot(picks=[ch], baseline=(-0.2, 0), mode='mean', axes=axes[idx], colorbar=False, show=False)
    axes[idx].set_title(ch)

# Hide any unused subplots
for ax in axes[n_channels:]:
    ax.axis('off')

plt.tight_layout()
fig.suptitle('Alpha Band TFR Power - Control (per channel)', y=1.02, fontsize=18)
plt.show()

In [ ]:
import numpy as np

# power.data shape: (n_channels, n_frequencies, n_times)
# Calculate mean across frequency and time for each channel (average alpha power per channel)
avg_alpha_per_channel = power.data.mean(axis=(1, 2))  # shape: (n_channels,)
# Calculate grand average alpha power across all channels
grand_avg_alpha = avg_alpha_per_channel.mean()

# Print results
for ch, avg in zip(power.ch_names, avg_alpha_per_channel):
    print(f"Channel {ch}: Average alpha power = {avg:.4e}")

print(f"\nGrand average alpha power across all channels: {grand_avg_alpha:.4e}")